# 爬蟲練習

## 郭耀仁

## 批踢踢實業坊

- 熱門看板
- 一般看板
- 十八禁看板

## 熱門看板

- 網址：<https://www.ptt.cc/bbs/hotboards.html>
- 先練習將第一欄：版名抓出來

In [1]:
library(rvest)

url <- "https://www.ptt.cc/bbs/hotboards.html"
raw_html <- read_html(url)
board_css <- ".board-name"
boards <- raw_html %>%
    html_nodes(css = board_css) %>%
    html_text()
head(boards)
length(boards)

Loading required package: xml2


[1] "Gossiping" "NBA"       "Baseball"  "Stock"     "C_Chat"    "LoL"

[1] 128

## 熱門看板（2）

- 練習將第二欄：人氣抓出來

```r
library(rvest)

url <- "https://www.ptt.cc/bbs/hotboards.html"
raw_html <- read_html(url)
viewer_css <- "___"
viewers <- raw_html %>%
    html_nodes(css = viewer_css) %>%
    html_text()
head(viewers)
length(viewers)
```

## 熱門看板（3）

- 是否覺得跟第一個欄位的程式很像？
- 我們只需要更換 CSS Selector 或 XPath 而已
- 寫在 for 迴圈裡面，然後用一個函數包起來：

In [2]:
library(rvest)

ptt_hot_boards_crawler <- function(){
  url <- "https://www.ptt.cc/bbs/hotboards.html"
  raw_html <- read_html(url)
  
  board_xpath <- "//a[@class='board']/div[@class='board-name']"
  viewers_xpath <- "//a[@class='board']/div[@class='board-nuser']/span"
  category_xpath <- "//a[@class='board']/div[@class='board-class']"
  title_xpath <- "//a[@class='board']/div[@class='board-title']"
  
  hot_boards_columns <- list()
  columns <- c(board_xpath, viewers_xpath, category_xpath, title_xpath)
  
  for (i in 1:length(columns)){
    content <- raw_html %>%
      html_nodes(xpath = columns[i]) %>%
      html_text()
    hot_boards_columns[[i]] <- content
  }
  names(hot_boards_columns) <- c("board", "viewers", "category", "title")
  return(hot_boards_columns)
}

## 熱門看板（4）

- `ptt_hot_boards_cralwer()` 函數不需要輸入，可以直接呼叫
- 輸入是一個元素有命名清單（list）

In [3]:
current_ptt_hot_boards <- ptt_hot_boards_crawler()
head(current_ptt_hot_boards$board)
head(current_ptt_hot_boards$viewers)
head(current_ptt_hot_boards$category)
head(current_ptt_hot_boards$title)

[1] "Gossiping" "NBA"       "Baseball"  "Stock"     "C_Chat"    "LoL"

[1] "17246" "7981"  "3839"  "2464"  "2416"  "2267"

[1] "綜合" "籃球" "棒球" "學術" "聊天" "遊戲"

[1] "◎[八卦] 夏天熱肥宅"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
[2] "◎[NBA]總冠軍賽6/8 9:00 詠琪大戰"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
[3] "◎[棒球] YOH來了   [email protected]/* <![CDATA[ */!function(t,e,r,n,c,a,p){try{t=document.currentScript||function(){for(t=document.getElementsByTagName('script'),e=t.length;e--;)if(t[e].getAttribute('data-cfhash'))return t[e]}();if(t&&(c=t.previousSibling)){p=t.parentNode;if(a=c.getAttribute('data-cfemail')){for(e='',r='0x'+a.substr(0,2)|0,n=2;a.length-n;n+=2)e+='%'+('0'+('0x'+a.substr(n,2)^r).toString(16)).slice(-2);p.replaceChild(document.createTextNode(decodeURIComponent(e)),c)}p.removeChild(t)}}catch(u){}}()/* ]]> */"
[4] "◎[股板] 徵文比賽中 好文連載中"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
[5] "◎[希洽] 2016希洽人氣動畫提名中！"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
[6] "◎[LoL] 大隊板時代板務說明見置底"

## 熱門看板（5）

- 練習將熱門看板的連結抓出來
- 提示：
    - css selector 改用 `.board`
    - 改用 `html_attr("href")` 而非 `html_text()`

## 一般看板

- 以 NBA 為例：<https://www.ptt.cc/bbs/NBA/index.html>
- 要怎麼將一整頁的文章內容都抓出來呢？

## 一般看板（2）

- 先把看得到的抓出來：
    - 推文數
    - 文章標題
    - ID

In [4]:
library(rvest)

url <- "https://www.ptt.cc/bbs/NBA/index.html"
raw_html <- read_html(url)

nrec_xpath <- "//div[@class='nrec']"
title_xpath <- "//div[@class='title']"
id_xpath <- "//div[@class='meta']/div[@class='author']"

article_info <- list()
columns <- c(nrec_xpath, title_xpath, id_xpath)

for (i in 1:length(columns)){
  content <- raw_html %>%
    html_nodes(xpath = columns[i]) %>%
    html_text()
  article_info[[i]] <- content
}

names(article_info) <- c("nrec", "title", "author_id")

## 一般看板（3）

- 接著我們要清理一下文章標題

In [5]:
head(article_info$title)
head(gsub(pattern = "\n\t+\n\t+", article_info$title, replacement = ""))

[1] "\n\t\t\t\n\t\t\t\t[討論] 騎士隊是不是真的故意輸塞爾提克的\n\t\t\t\n\t\t\t"     
[2] "\n\t\t\t\n\t\t\t\t[情報] LBJ：如果再來一次，我還會把球傳給KK\n\t\t\t\n\t\t\t"  
[3] "\n\t\t\t\n\t\t\t\t[新聞] 最強勇士誰能擋？活塞名將：我們7場搞定\n\t\t\t\n\t\t\t"
[4] "\n\t\t\t\n\t\t\t\t[討論] 如果騎士逆轉冠軍.姆斯的歷史評價\n\t\t\t\n\t\t\t"      
[5] "\n\t\t\t\n\t\t\t\t[討論] 騎士當初為什麼要釋出Liggens？\n\t\t\t\n\t\t\t"        
[6] "\n\t\t\t\n\t\t\t\tRe: [情報] Final MVP 即時賠率\n\t\t\t\n\t\t\t"

[1] "[討論] 騎士隊是不是真的故意輸塞爾提克的"     
[2] "[情報] LBJ：如果再來一次，我還會把球傳給KK"  
[3] "[新聞] 最強勇士誰能擋？活塞名將：我們7場搞定"
[4] "[討論] 如果騎士逆轉冠軍.姆斯的歷史評價"      
[5] "[討論] 騎士當初為什麼要釋出Liggens？"        
[6] "Re: [情報] Final MVP 即時賠率"

## 一般看板（4）

- 隨便點一篇文章進去看
- 我們要抓文章的哪些內容？
    - 作者
    - 標題
    - 時間
    - 內文
    - IP
    - 推文標記
    - 推文 ID
    - 推文內容
    - 推文時間

In [6]:
library(rvest)

article_detail <- function(url){
    raw_html <- read_html(url)
    
    author_css <- ".article-metaline:nth-child(1) .article-meta-value"
    title_css <- ".article-metaline-right+ .article-metaline .article-meta-value"
    time_css <- ".article-metaline+ .article-metaline .article-meta-value"
    main_content_css <- "#main-content"
    ip_css <- ".hl+ .f2"
    push_css <- ".push-tag"
    push_id_css <- ".push-userid"
    push_content_css <- ".push-content"
    push_time_css <- ".push-ipdatetime"
    
    article_detail_info <- list()
    columns <- c(author_css, title_css, time_css, main_content_css, ip_css, push_css, push_id_css, push_content_css, push_time_css)
    for (i in 1:length(columns)){
        article_content <- raw_html %>%
            html_nodes(css = columns[i]) %>%
            html_text()
        article_detail_info[[i]] <- article_content
    }
    names(article_detail_info) <- c("author", "title", "time", "main_content", "ip", "push", "push_id", "push_content", "push_time")
    return(article_detail_info)
}

In [7]:
article_url <- "https://www.ptt.cc/bbs/NBA/M.1496931569.A.682.html"
nba_article_details <- article_detail(article_url)

## 一般看板（5）

- 清理內文
- 清理IP
- 清理推文
- 清理推文 ID
- 清理推文內容
- 清理推文時間

## 一般看板（6）

- 清理內文

In [8]:
nba_article_details$main_content <- nba_article_details$main_content %>%
    gsub(pattern = "\n", ., replacement = "") %>% # 清理斷行符號
    gsub(pattern = "--.+", ., replacement = "") %>% # 去尾
    gsub(pattern = "作者.+:[0-9]{2}\\s[0-9]{4}", ., replacement = "") # 去頭

## 一般看板（7）

- 清理IP

In [9]:
ip_start <- regexpr(pattern = "[0-9]+", nba_article_details$ip)
ip_end <- nchar(nba_article_details$ip)
nba_article_details$ip <- substr(nba_article_details$ip, start = ip_start, stop = ip_end)
nba_article_details$ip <- gsub(pattern = "\n", nba_article_details$ip, replacement = "") # 清理斷行符號

## 一般看板（8）

- 清理推文
- 清理推文 ID
- 清理推文內容
- 清理推文時間

In [10]:
nba_article_details$push <- gsub(pattern = "\\s", nba_article_details$push, replacement = "")
nba_article_details$push_id <- gsub(pattern = "\\s", nba_article_details$push_id, replacement = "")
nba_article_details$push_content <- nba_article_details$push_content %>%
    gsub(pattern = "\\s", ., replacement = "") %>%
    gsub(pattern = ":", ., replacement = "")
nba_article_details$push_time <- nba_article_details$push_time %>%
    gsub(pattern = "^\\s", ., replacement = "") %>%
    gsub(pattern = "\n", ., replacement = "")

## 一般看板（8）

- 最後我們把清理資料的程式加入原本的 `article_detail()` 函數中

In [11]:
library(rvest)

article_detail <- function(url){
    raw_html <- read_html(url)
    
    author_css <- ".article-metaline:nth-child(1) .article-meta-value"
    title_css <- ".article-metaline-right+ .article-metaline .article-meta-value"
    time_css <- ".article-metaline+ .article-metaline .article-meta-value"
    main_content_css <- "#main-content"
    ip_css <- ".hl+ .f2"
    push_css <- ".push-tag"
    push_id_css <- ".push-userid"
    push_content_css <- ".push-content"
    push_time_css <- ".push-ipdatetime"
    
    article_detail_info <- list()
    columns <- c(author_css, title_css, time_css, main_content_css, ip_css, push_css, push_id_css, push_content_css, push_time_css)
    for (i in 1:length(columns)){
        article_content <- raw_html %>%
            html_nodes(css = columns[i]) %>%
            html_text()
        article_detail_info[[i]] <- article_content
    }
    
    names(article_detail_info) <- c("author", "title", "time", "main_content", "ip", "push", "push_id", "push_content", "push_time")
    
    article_detail_info$main_content <- article_detail_info$main_content %>%
        gsub(pattern = "\n", ., replacement = "") %>% # 清理斷行符號
        gsub(pattern = "--.+", ., replacement = "") %>% # 去尾
        gsub(pattern = "作者.+:[0-9]{2}\\s[0-9]{4}", ., replacement = "") # 去頭
    ip_start <- regexpr(pattern = "[0-9]+", article_detail_info$ip)
    ip_end <- nchar(article_detail_info$ip)
    article_detail_info$ip <- substr(article_detail_info$ip, start = ip_start, stop = ip_end)
    article_detail_info$ip <- gsub(pattern = "\n", article_detail_info$ip, replacement = "") # 清理斷行符號
    article_detail_info$push <- gsub(pattern = "\\s", article_detail_info$push, replacement = "")
    article_detail_info$push_id <- gsub(pattern = "\\s", article_detail_info$push_id, replacement = "")
    article_detail_info$push_content <- article_detail_info$push_content %>%
        gsub(pattern = "\\s", ., replacement = "") %>%
        gsub(pattern = ":", ., replacement = "")
    article_detail_info$push_time <- article_detail_info$push_time %>%
        gsub(pattern = "^\\s", ., replacement = "") %>%
        gsub(pattern = "\n", ., replacement = "")
    
    return(article_detail_info)
}

## 一般看板（9）

- 接著我們回到最新一頁將所有文章連結抓出來

In [14]:
library(rvest)

url <- "https://www.ptt.cc/bbs/NBA/index.html"
raw_html <- read_html(url)

article_link_xpath <- "//div[@class='title']/a"
article_links <- raw_html %>%
    html_nodes(xpath = article_link_xpath) %>%
    html_attr("href")
article_links # relative url
article_links <- paste("https://www.ptt.cc", article_links, sep = "")
article_links # absolute url

[1] "/bbs/NBA/M.1496978847.A.0E5.html" "/bbs/NBA/M.1496979049.A.15E.html"
 [3] "/bbs/NBA/M.1496979276.A.1F7.html" "/bbs/NBA/M.1496979773.A.3F5.html"
 [5] "/bbs/NBA/M.1496979977.A.781.html" "/bbs/NBA/M.1496979999.A.7B9.html"
 [7] "/bbs/NBA/M.1496980225.A.A06.html" "/bbs/NBA/M.1496980264.A.FFB.html"
 [9] "/bbs/NBA/M.1496980420.A.E39.html" "/bbs/NBA/M.1496980542.A.7A4.html"
[11] "/bbs/NBA/M.1496981408.A.9EE.html" "/bbs/NBA/M.1496981524.A.42E.html"
[13] "/bbs/NBA/M.1496981589.A.D64.html" "/bbs/NBA/M.1496981782.A.EC0.html"
[15] "/bbs/NBA/M.1496982384.A.D4D.html" "/bbs/NBA/M.1496982548.A.139.html"
[17] "/bbs/NBA/M.1496982658.A.37A.html" "/bbs/NBA/M.1496983106.A.1C3.html"
[19] "/bbs/NBA/M.1496983475.A.800.html" "/bbs/NBA/M.1496983561.A.7F3.html"
[21] "/bbs/NBA/M.1480648734.A.D28.html" "/bbs/NBA/M.1478497499.A.150.html"
[23] "/bbs/NBA/M.1486126665.A.D3C.html" "/bbs/NBA/M.1496893864.A.9E3.html"

[1] "https://www.ptt.cc/bbs/NBA/M.1496978847.A.0E5.html"
 [2] "https://www.ptt.cc/bbs/NBA/M.1496979049.A.15E.html"
 [3] "https://www.ptt.cc/bbs/NBA/M.1496979276.A.1F7.html"
 [4] "https://www.ptt.cc/bbs/NBA/M.1496979773.A.3F5.html"
 [5] "https://www.ptt.cc/bbs/NBA/M.1496979977.A.781.html"
 [6] "https://www.ptt.cc/bbs/NBA/M.1496979999.A.7B9.html"
 [7] "https://www.ptt.cc/bbs/NBA/M.1496980225.A.A06.html"
 [8] "https://www.ptt.cc/bbs/NBA/M.1496980264.A.FFB.html"
 [9] "https://www.ptt.cc/bbs/NBA/M.1496980420.A.E39.html"
[10] "https://www.ptt.cc/bbs/NBA/M.1496980542.A.7A4.html"
[11] "https://www.ptt.cc/bbs/NBA/M.1496981408.A.9EE.html"
[12] "https://www.ptt.cc/bbs/NBA/M.1496981524.A.42E.html"
[13] "https://www.ptt.cc/bbs/NBA/M.1496981589.A.D64.html"
[14] "https://www.ptt.cc/bbs/NBA/M.1496981782.A.EC0.html"
[15] "https://www.ptt.cc/bbs/NBA/M.1496982384.A.D4D.html"
[16] "https://www.ptt.cc/bbs/NBA/M.1496982548.A.139.html"
[17] "https://www.ptt.cc/bbs/NBA/M.1496982658.A.37A.html"
[18] "https://www.ptt.cc/bbs/NBA/M.1496983106.A.1C3.html"
[19] "https://www.ptt.cc/bbs/NBA/M.1496983475.A.800.html"
[20] "https://www.ptt.cc/bbs/NBA/M.1496983561.A.7F3.html"
[21] "https://www.ptt.cc/bbs/NBA/M.1480648734.A.D28.html"
[22] "https://www.ptt.cc/bbs/NBA/M.1478497499.A.150.html"
[23] "https://www.ptt.cc/bbs/NBA/M.1486126665.A.D3C.html"
[24] "https://www.ptt.cc/bbs/NBA/M.1496893864.A.9E3.html"

## 一般看板（10）

- 一篇文章的詳細內容存在一個清單中
- 外面再用一個清單儲存這些清單（Nested lists）

In [15]:
article_lists <- list()
for (i in 1:length(article_links)){
    article_lists[[i]] <- article_detail(article_links[i])
}

## 十八禁看板

- 十八禁看板會自動跳轉一個確認是否滿十八歲的頁面
- 我們以八卦版（Gossiping）為例：<https://www.ptt.cc/bbs/Gossiping/index.html>

![](img/ch0801.png)

## 十八禁看板（2）

- 瀏覽器的 session 會將使用者點選**我同意**的資訊存在 cookie 中
- 下載一個 Chrome 外掛來觀察：[EditThisCookie](https://chrome.google.com/webstore/search/editthiscookie?hl=zh-TW)

## 十八禁看板（3）

- 點選之後就如同進入一般看板
- 打開 EditThisCookie 外掛可以看到

![](img/ch0802.png)

## 十八禁看板（4）

- 接著我們按上一頁，再連回八卦版
- 這時就沒有跳轉一個確認是否滿十八歲的頁面
- 這是因為 cookie（over18 == 1）已經儲存在 session 中的關係

## 十八禁看板（5）

- 利用 EditThisCookie 將它刪除
- 接著我們按上一頁，再連回八卦版
- 這時就跟我們先前一樣會跳轉一個確認是否滿十八歲的頁面

## 十八禁看板（6）

- 回到 R 語言，如果我們將十八禁看板的網址直接傳入
- 將會因為跳轉頁面的關係無法到達看板的最新文章頁面

In [ ]:
url <- "https://www.ptt.cc/bbs/Gossiping/index.html"
ptt_gossiping <- readLines(url)

## 十八禁看板（7）

- 這時我們要使用 `httr` 套件所提供的 `GET()` 函數幫忙設定 cookie

```
install.packages("httr")
```

In [ ]:
# install.packages("httr")
library(httr)

url <- "https://www.ptt.cc/bbs/Gossiping/index.html"
ptt_gossiping <- GET(url, set_cookies(over18 = 1))

## 十八禁看板（8）

- 在一般看板的程式中加入 `httr` 的 `GET()` 函數即可

In [ ]:
library(rvest)
library(httr)

ptt_index_crawler <- function(url){
    ptt_index_html_doc <- url %>%
        GET(set_cookies(over18 = 1)) %>%
        read_html()
    nrec_xpath <- "//div[@class='nrec']"
    title_xpath <- "//div[@class='title']"
    id_xpath <- "//div[@class='meta']/div[@class='author']"
    article_info <- list()
    columns <- c(nrec_xpath, title_xpath, id_xpath)
    for (i in 1:length(columns)){
        content <- ptt_index_html_doc %>%
            html_nodes(xpath = columns[i]) %>%
            html_text()
        article_info[[i]] <- content
    }
    names(article_info) <- c("nrec", "title", "author_id")
    article_info$title <- gsub(pattern = "\n\t+\n\t+", article_info$title, replacement = "")
    return(article_info)
}

ptt_gossiping_index <- ptt_index_crawler("https://www.ptt.cc/bbs/Gossiping/index.html")
ptt_nba_index <- ptt_index_crawler("https://www.ptt.cc/bbs/NBA/index.html")